# Continuous SIMP example

## Commented Program

What follows is a program spliced with comments.

In [1]:
using TopOpt

### Define the problem

In [2]:
E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0 # downward force

problems = Any[
    PointLoadCantilever(Val{:Linear}, (60, 20, 20), (1.0, 1.0, 1.0), E, v, f),
    PointLoadCantilever(Val{:Linear}, (160, 40), (1.0, 1.0), E, v, f),
    HalfMBB(Val{:Linear}, (60, 20), (1.0, 1.0), E, v, f),
    LBeam(Val{:Linear}, Float64; force=f),
    TieBeam(Val{:Quadratic}, Float64),
]
problem_names = [
    "3d cantilever beam", "cantilever beam", "half MBB beam", "L-beam", "tie-beam"
]

i = 2
println(problem_names[i])
problem = problems[i]

cantilever beam


### Parameter settings

In [3]:
V = 0.5 # volume fraction
xmin = 0.001 # minimum density
rmin = 3.0
steps = 40 # maximum number of penalty steps, delta_p0 = 0.1
reuse = true # adaptive penalty flag

convcriteria = Nonconvex.GenericCriteria()
penalty = TopOpt.PowerPenalty(1.0)
pcont = Continuation(penalty; steps=steps, xmin=xmin, pmax=5.0)

(::PowerContinuation{Float64}) (generic function with 1 method)

NOTE: non-convexity + computational error lead to different solutions that satisfy the KKT tolerance

In [4]:
mma_options = options = MMAOptions(; maxiter=1000)
maxtol = 0.01 # maximum tolerance
mintol = 0.0001 # minimum tolerance
b = log(mintol / maxtol) / steps
a = maxtol / exp(b)
mma_options_gen = TopOpt.MMAOptionsGen(;
    steps=steps,
    initial_options=mma_options,
    ftol_gen=ExponentialContinuation(a, b, 0.0, steps + 1, mintol),
)
csimp_options = TopOpt.CSIMPOptions(;
    steps=steps, options_gen=mma_options_gen, p_gen=pcont, reuse=reuse
)

### Define a finite element solver

In [5]:
solver = FEASolver(Direct, problem; xmin=xmin, penalty=penalty)

### Define compliance objective

In [6]:
comp = Compliance(problem, solver)
filter = if problem isa TopOptProblems.TieBeam
    identity
else
    DensityFilter(solver; rmin=rmin)
end
obj = x -> comp(filter(x))

#1 (generic function with 1 method)

### Define volume constraint

In [7]:
volfrac = TopOpt.Volume(problem, solver)
constr = x -> volfrac(filter(x)) - V

#3 (generic function with 1 method)

### Define subproblem optimizer

In [8]:
x0 = fill(V, length(solver.vars))
optimizer = Optimizer(
    obj, constr, x0, MMA87(); options=mma_options, convcriteria=convcriteria
)

Internal error: encountered unexpected error in runtime:
TypeError(func=:<:, context="", expected=Type, got=TApprox<:NonconvexMMA.AbstractMMAApprox)
jl_type_error_rt at /buildworker/worker/package_linux64/build/src/rtutils.c:119
jl_type_error at /buildworker/worker/package_linux64/build/src/rtutils.c:127
jl_f_issubtype at /buildworker/worker/package_linux64/build/src/builtins.c:491
apply_type_nothrow at ./compiler/tfuncs.jl:1245
_builtin_nothrow at ./compiler/tfuncs.jl:1566
stmt_effect_free at ./compiler/ssair/queries.jl:36
check_effect_free! at ./compiler/ssair/inlining.jl:1128 [inlined]
process_simple! at ./compiler/ssair/inlining.jl:1185
assemble_inline_todo! at ./compiler/ssair/inlining.jl:1358
ssa_inlining_pass! at ./compiler/ssair/inlining.jl:80
jfptr_ssa_inlining_passNOT._8869.clone_1 at /opt/hostedtoolcache/julia/1.7.1/x64/lib/julia/sys.so (unknown line)
_jl_invoke at /buildworker/worker/package_linux64/build/src/gf.c:2247 [inlined]
jl_apply_generic at /buildworker/worker/packa

### Define continuation SIMP optimizer

In [9]:
simp = SIMP(optimizer, solver, penalty.p)
cont_simp = ContinuationSIMP(simp, steps, csimp_options)

### Solve

In [10]:
result = cont_simp(x0)

@show result.convstate
@show optimizer.workspace.iter
@show result.objval

result.convstate = NonconvexCore.ConvergenceState{Float64}
  Δx: Float64 0.19890886735275043
  Δf: Float64 2.33965716686248
  relΔf: Float64 0.007587785984346274
  kkt_residual: Float64 0.42771888549275544
  ipopt_residual: Float64 0.42771888549275544
  infeas: Float64 0.0
  x_converged: Bool false
  fabs_converged: Bool false
  frel_converged: Bool true
  kkt_converged: Bool false
  ipopt_converged: Bool false
  infeas_converged: Bool true
  f_increased: Bool false
  converged: Bool true

optimizer.workspace.iter = 5
result.objval = 308.33517100103126


308.33517100103126

### (Optional) Visualize the result using Makie.jl
Need to run `using Pkg; Pkg.add(Makie)` first
```julia
using TopOpt.TopOptProblems.Visualization: visualize
fig = visualize(problem; topology = result.topology,
    problem; topology = result.topology, default_exagg_scale = 0.07,
    scale_range = 10.0, vector_linewidth = 3, vector_arrowsize = 0.5,
)
Makie.display(fig)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*